In [ ]:
!pip install openai numpy pandas singlestoredb langchain==0.1.8 langchain-community==0.0.21 langchain-core==0.1.25 langchain-openai==0.0.6 faiss-cpu --quiet


In [12]:
import os

os.environ['OPENAI_API_KEY'] = 'Your APi Key here'

In [ ]:

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from openai import OpenAI
import os
import pandas as pd
import requests


# First, download the text file content from the URLSS
file_url = "https://sherlock-holm.es/stories/plain-text/stud.txt"
response = requests.get(file_url)
if response.status_code == 200:
    file_content = response.text
    file_path = 'local_example.txt'  # Save the content to a local file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(file_content)
else:
    raise Exception("Failed to download the file.")

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
if not client.api_key:
    raise Exception("OPENAI_API_KEY environment variable not set. Please set it before running the script.")

# Initialize the OpenAI client with the API key
#openai.api_key = OPENAI_KEY

# Load and process documents from the local file
loader = TextLoader(file_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Initialize embeddings and create a document search database
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(docs, embeddings)

# Chat loop
while True:
    user_query = input("\nYou: ")
    if user_query.lower() in ['quit', 'exit']:
        print("Exiting chatbot.")
        break

    docs = docsearch.similarity_search(user_query)
    if docs:
        context = docs[0].page_content
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_query},
                {"role": "assistant", "content": context}
            ],
            temperature=0.7,
            max_tokens=150,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        print("AI:", response.choices[0].message.content.strip())
    else:
        print("AI: Sorry, I couldn't find relevant information.")



You:  what did Shakespeare say?


AI: It seems like you may have pasted a passage from a Sherlock Holmes story by Sir Arthur Conan Doyle instead of a quote from Shakespeare. If you're looking for a specific quote from Shakespeare, please let me know so I can assist you further.



You:  I gave him a short sketch of my adventures, and had hardly concluded      it by the time that we reached our destination.


AI: It sounds like you had quite an intriguing and eventful journey with your companion. The way you describe the scene with Patent-leathers and Square-toes is quite vivid and captivating. It seems like there is a mystery unfolding, with clues hidden in the dust and the behavior of the characters. I hope your adventures continue to be exciting and full of intrigue. If you need any assistance or advice along the way, feel free to ask!



You:  who are you?


...

AI: It appears that you may have misspelled the name of the famous playwright and poet William Shakespeare as "shake sphere." William Shakespeare, often referred to simply as Shakespeare, was an English playwright and poet who is widely regarded as one of the greatest writers in the English language. He lived in the late 16th and early 17th centuries and is known for his plays such as "Romeo and Juliet," "Hamlet," and "Macbeth." If you have any specific questions about Shakespeare or his works, feel free to ask!



You:  "And who was the first?" I asked.


AI: I'm sorry, it seems like you're referencing a specific text or conversation that I'm not familiar with. Can you provide more context or clarify your question so I can better assist you?



You:  "A fellow who is working at the chemical laboratory up at the      hospital. He was bemoaning himself this morning because he could not      get someone to go halves with him in some nice rooms which he had      found, and which were too much for his purse."


AI: It seems like you are quoting a passage from Sir Arthur Conan Doyle's "A Study in Scarlet," which introduces the characters of Dr. John Watson and Sherlock Holmes. The dialogue captures the initial meeting between Watson and Holmes, as well as Watson's curiosity about his potential future roommate. If you have any specific questions or need further assistance with this text, feel free to ask.
